<a href="https://colab.research.google.com/github/shubhamgoel90/Data-Modeling/blob/master/Crop_AI_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
# Install Kaggle library
!pip install -q kaggle

In [ ]:
# Upload kaggle API key file
uploaded = files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d emmarex/plantdisease

In [ ]:
%%capture
!unzip plantdisease.zip

In [ ]:
!ls

In [ ]:
!ls PlantVillage

In [ ]:
!ls plantvillage

In [2]:
import os
import pickle
folders = os.listdir("PlantVillage")
print(folders)

['Tomato_Bacterial_spot', 'Potato___Early_blight', 'Tomato__Target_Spot', 'Pepper__bell___Bacterial_spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Potato___Late_blight', 'Tomato_Late_blight', 'Pepper__bell___healthy', 'Tomato_Leaf_Mold', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato__Tomato_mosaic_virus', 'Potato___healthy', 'Tomato_Septoria_leaf_spot']


In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_gen.flow_from_directory("PlantVillage", target_size = (224,224), batch_size = 64, class_mode = "categorical", shuffle = True)
class_dictionary = train_generator.class_indices
class_dictionary
pickle.dump(class_dictionary, open("encoder.pkl", "wb"))

Found 20638 images belonging to 15 classes.


# New Section

In [4]:
train_generator.labels

array([ 0,  0,  0, ..., 14, 14, 14], dtype=int32)

In [5]:
x, y = train_generator.next()
print(x.shape)
print(y.shape)

(64, 224, 224, 3)
(64, 15)


In [6]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model


In [7]:
model = ResNet50(include_top =False, weights = "imagenet", input_shape = (224,224,3))

/home/vaibagga/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [8]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256 , activation = "relu")(av1)
d1 = Dropout(0.4)(fc1)
fc2 = Dense(15 , activation= "softmax")(d1)

model_new = Model(input = model.input, output = fc2)

/home/vaibagga/anaconda3/envs/cropai/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [9]:
adam = Adam(lr = 0.00003)
model_new.compile(loss = "categorical_crossentropy", optimizer ="adam", metrics = ["accuracy"] )

In [10]:
for ix in range(169):
    model_new.layers[ix].trainable = False
model_new.compile(loss = "categorical_crossentropy", optimizer ="adam", metrics = ["accuracy"] )


In [14]:
hist = model_new.fit(train_generator, epochs = 5, steps_per_epoch= 325)

model_new.save("CropAI_model_train_gen_epoch_5.h5")
print("model weights are save on disk")

Epoch 1/5
 16/325 [>.............................] - ETA: 48:26 - loss: 0.0279 - accuracy: 0.9932

KeyboardInterrupt: 